In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# import dataset
df = pd.read_excel("restoran.xlsx")

# lihat data
df

In [ ]:
def plot_linguistik(titik, warna, label='', min=0, max=1):
    # nilai y selalu 0, 1, 1, 0 untuk membentu
    y = [0, 1, 1, 0]
 
    # plot garis pertama
    plt.plot([min, titik[0]], [0, 0], warna)
    
    # plot garis antara
    for i in range(len(titik) - 1):
        plt.plot([titik[i], titik[i + 1]], [y[i], y[i + 1]], warna)
    
    # plot garis terakhir
    plt.plot([titik[3], max], [0, 0], warna, label=label)

In [ ]:
def plot_membership(membership, min=0, max=1):
    # warna yang akan digunakan
    warna = ['r', 'g', 'b', 'c']
 
    # plot tiap linguistik
    i = 0
    for ling in membership:
        plot_linguistik(membership[ling], warna[i], ling, min=min, max=max)
        i += 1
    
    plt.legend(loc=3)
    plt.show()

In [ ]:
def fuzzy_linguistik(x, titik):


    # titik berada didepan dan dibelakang grafik
    if x < titik[0] or x > titik[3]:
        return 0
    
    # titik berada ditengah grafik
    elif x >= titik[1] and x <= titik[2]:
        return 1
    
    # titik berada di 1/4 grafik kiri
    elif x >= titik[0] and x < titik[1]:
        return (x - titik[0]) / (titik[1] - titik[0])
    
    # titik berada di 1/4 grafik kanan
    elif x >= titik[2] and x < titik[3]:
        return (titik[3] - x) / (titik[3] - titik[2])

    return 0

In [ ]:
def fuzzy_membership(x, membership):

    hasil = {}
    # cari nilai fuzzy untuk masing-masing linguistik dalam membership
    for ling in membership:
        hasil[ling] = fuzzy_linguistik(x, membership[ling])

    return hasil

In [ ]:
def fuzzyfikasi(x, list_membership):
    hasil = []
    # menghitung seluruh hasil fuzzifikasi dari seluruh membership
    for i in range(len(list_membership)):
        hasil.append(fuzzy_membership(x[i], list_membership[i]))

    return hasil

In [ ]:
def inferensi(x_fuzzy, rule):
    hasil = {}
    for i1, j1 in x_fuzzy[0].items():
        for i2, j2 in x_fuzzy[1].items():
            min_value = min(j1, j2)
            pointer = (i1, i2)
            value = hasil.get(rule[pointer], 0)
            if min(j1, j2) < value :
                hasil[rule[pointer]] = value
            else :
                if j1 < j2 :
                    hasil[rule[pointer]] = j1
                else :
                    hasil[rule[pointer]] = j2
    return hasilz

In [ ]:
def defuzz_sugeno(x_infer, membership):

    num, den = 0, 0

    # menghitung seluruh hasil dari pembilang dan penyebut
    for ling in membership:
        num += x_infer[ling] * membership[ling]
        den += x_infer[ling]
    if den != 0 :
        return num / den
    else :
        return 0

In [ ]:
# membership input pelayanan
pelayanan = {
    'Dissatisfied'    : [ 0,  0,  30,  50],
    'Neutral'         : [ 45,  55,  65,  80],
    'Satisfied'       : [ 75, 85, 100, 100]
}

# membership input makanan
makanan = {
    'Bad'   : [0,  0,  2,  4],
    'Okay'  : [3.5,  4,  5.5,  6],
    'Good'  : [6.5, 7, 7.5, 8],
    'Tasty' : [7.75, 8.5, 10, 10]
}

# plot seluruh membership
plot_membership(pelayanan, max=100)
plot_membership(makanan, max=10)

# rule inferensi
rule = {
    ('Dissatisfied',  'Bad')     : 'poor',
    ('Dissatisfied',  'Okay')     : 'poor',
    ('Dissatisfied',  'Good')    : 'Average',
    ('Dissatisfied',  'Tasty')    : 'Average',
    ('Neutral',  'Bad')     : 'poor',
    ('Neutral',  'Okay')     : 'Average',
    ('Neutral',  'Good')    : 'Average',
    ('Neutral',  'Tasty')    : 'Good',
    ('Satisfied', 'Bad')     : 'poor',
    ('Satisfied', 'Okay')     : 'average',
    ('Satisfied', 'Good')    : 'Good',
    ('Satisfied', 'Tasty')    : 'Excellent',
}

# membership sugeno
accept_sugeno = {'Excellent':100, 'Good':80, 'Average':60, 'poor':30}

# plot membership sugeno
colors = 'rgbc'
for key, val in accept_sugeno.items():
    plt.plot([val, val], [0, 1], colors[0], label=key )
    colors = colors[1:]
plt.legend(loc=3)

plt.show()

In [ ]:
# lakukan seluruh proses fuzzy pada dataset
skor = []
for x in df.drop('id', axis=1).values:
    x_fuzzy = fuzzyfikasi(x, [pelayanan, makanan])
    x_infer = inferensi(x_fuzzy, rule)
    x_defuz = defuzz_sugeno(x_infer, accept_sugeno)
    skor.append(x_defuz)

# append hasilnya ke dataset
df['Skor'] = skor

# simpan ranking restaurant
rank_restaurant = df.sort_values(by='Skor', ascending=False)[:10]
df_not_restaurant= df.drop(rank_restaurant.index)
plt.scatter(rank_restaurant.pelayanan, rank_restaurant.makanan, label = "Best Restaurant")
plt.scatter(df_not_restaurant.pelayanan,df_not_restaurant.makanan, label='Not Best Restaurant')

# beri informasi tambahan pada grafik
plt.xlabel('Pelayanan')
plt.ylabel('Makanan')
plt.title('Data Restaurant')
plt.show()

In [ ]:
rank_restaurant = rank_restaurant.reset_index(drop=True)
rank_restaurant.insert(0, "peringkat", [1,2,3,4,5,6,7,8,9,10], False)
rank_restaurant = rank_restaurant.set_index("peringkat")
rank_restaurant

In [ ]:

rank_restaurant["id"]

In [ ]:
out_file = rank_restaurant.id
out_file.to_excel("peringkat.xlsx", index=False)

In [ ]:
out_file.to_excel("peringkat.xlsx", index=False)